# Data cleanup

Sometimes MATLAB writes individual entries into the localization files that are not easily parsed (or impossible to parse in ThunderSTORM). For this reason, I need to find a way to clean up the data files.

### Clean up of data

In [1]:
%pylab
import DataSTORM.processors as ds
import pandas as pd
from pathlib import Path

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
filename = Path('../test-data/MicroTubules_LargeFOV/FOV1_1500_10ms_1_MMStack_locResults.dat')
with open(str(filename.resolve()), 'r') as file:
    df = pd.read_csv(file, engine = 'c')

/home/kmdouglass/anaconda3/envs/DataSTORM/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Unfortunately, the data in the uncertainty column was saved such that most of the numbers are floats, but some are strings representing floats and some are in a strange complex exponential form. Let's filter out these rows to make working with the data frame easier and to protoype a clean up routine.

First, I'll generate a mask to pick out the rows containing strings.

In [3]:
stringMask = df['uncertainty [nm]'].map(lambda x: isinstance(x, str)).as_matrix()

Let's see what the strings look like:

In [ ]:
df['uncertainty [nm]'][stringMask]

In [25]:
import importlib
importlib.reload(ds)

<module 'DataSTORM.processors' from '/home/kmdouglass/src/DataSTORM/DataSTORM/processors.py'>

In [26]:
# Try DataSTORM's cleanup command and compare to manual clean up later
clean = ds.CleanUp()
dfClean = clean(df)

In [27]:
dfClean.describe()

,x [nm],y [nm],z [nm],frame,uncertainty [nm],intensity [photon],offset [photon],loglikelihood,sigma [nm]
count,8667397.000000,8667397.000000,8667397,8667397.000000,8.667397e+06,8667397.000000,8667397.000000,8667397.000000,8667397.000000
mean,34426.251067,34906.420891,0,20088.623120,8.055292e+11,3642.044266,344.474718,203.382508,134.408034
std,22145.547453,13027.020305,0,14416.319868,1.344216e+15,2569.207387,69.663961,824.384742,18.041903
min,7.470800,2.106100,0,100.000000,4.413800e-01,1.000000,98.010000,-29.974000,54.000000
25%,12235.000000,24933.000000,0,6720.000000,4.937900e+00,2256.800000,289.830000,96.946000,123.700000
50%,36195.000000,35067.000000,0,19463.000000,6.668600e+00,3023.200000,336.660000,124.960000,131.560000
75%,55460.000000,45292.000000,0,31084.000000,8.497100e+00,4289.300000,391.200000,174.810000,141.410000
max,67067.000000,67033.000000,0,49999.000000,3.560500e+18,86565.000000,2294.800000,454360.000000,378.000000


So, strangely there were about 11,000 localizations that were interpreted as strings in this data set. Let's cast them as numeric data types. Some of the strings cannot be recognized by the parser, so we'll convert those to NaN's by using the `errors='coerce'` argument.

In [4]:
df['uncertainty [nm]'] = pd.to_numeric(df['uncertainty [nm]'], errors='coerce')

Finally, we need to replace any Inf's with NaN's and then drop the NaN's. We'll reindex the final result.

In [5]:
df.replace([np.inf, -np.inf], np.nan, inplace = True)
df.dropna().describe()

,x [nm],y [nm],z [nm],frame,uncertainty [nm],intensity [photon],offset [photon],loglikelihood,sigma [nm]
count,8667397.000000,8667397.000000,8667397,8667397.000000,8.667397e+06,8667397.000000,8667397.000000,8667397.000000,8667397.000000
mean,34426.251067,34906.420891,0,20088.623120,8.055292e+11,3642.044266,344.474718,203.382508,134.408034
std,22145.547453,13027.020305,0,14416.319868,1.344216e+15,2569.207387,69.663961,824.384742,18.041903
min,7.470800,2.106100,0,100.000000,4.413800e-01,1.000000,98.010000,-29.974000,54.000000
25%,12235.000000,24933.000000,0,6720.000000,4.937900e+00,2256.800000,289.830000,96.946000,123.700000
50%,36195.000000,35067.000000,0,19463.000000,6.668600e+00,3023.200000,336.660000,124.960000,131.560000
75%,55460.000000,45292.000000,0,31084.000000,8.497100e+00,4289.300000,391.200000,174.810000,141.410000
max,67067.000000,67033.000000,0,49999.000000,3.560500e+18,86565.000000,2294.800000,454360.000000,378.000000


In [6]:
df.reindex()
print('Done.')

Done.


## Normal localization processing
Now that the data is cleaned up a bit, we'll proceed with our normal processing.

In [ ]:
df.describe()

In [ ]:
FilterLLR  = ds.Filter('loglikelihood', '<', 400)
FilterSig1 = ds.Filter('sigma [nm]',    '>', 100)
FilterSig2 = ds.Filter('sigma [nm]',    '<', 180)
df2        = FilterSig2(FilterSig1(FilterLLR(df)))

In [ ]:
df2.describe()

## Display the 2D histogram to visually identify fiducials

Now we need to make a 2D histogram to see whether the fiducial localizations are apparent.

In [1]:
import importlib
importlib.reload(ds)

NameError: name 'ds' is not defined

In [ ]:
# Find maximum x or y coordinate
maxPos    = np.max([df2['x [nm]'].max(), df2['y [nm]'].max()])
pixelSize = 100 # nm

numBins = int(maxPos / pixelSize)
plt.hist2d(df2['x [nm]'], df2['y [nm]'], bins = numBins)
plt.show()

In [ ]:
plt.close()

In [ ]:
corrector = ds.FiducialDriftCorrect(mergeRadius           = 50,
                                    offTime               = 1,
                                    minSegmentLength      = 20,
                                    minFracFiducialLength = 0.4,
                                    neighborRadius        = 500,
                                    smoothingWindowSize   = 625,
                                    smoothingFilterSize   = 500,
                                    searchRegions         = [{'xMin' :  2200,
                                                              'xMax' :  2800,
                                                              'yMin' : 33200,
                                                              'yMax' : 33700}])

In [ ]:
df3 = corrector(df2)

In [ ]:
# Find maximum x or y coordinate
maxPos    = np.max([df3['x [nm]'].max(), df3['y [nm]'].max()])
pixelSize = 100 # nm

numBins = int(maxPos / pixelSize)
plt.hist2d(df3['x [nm]'], df3['y [nm]'], bins = numBins)
plt.show()

In [ ]:
x0 = corrector.fiducialTrajectories[0]['x'].iloc[[0]].as_matrix()
plt.plot(corrector.fiducialTrajectories[0]['frame'], corrector.fiducialTrajectories[0]['x'] - x0)
plt.plot(corrector.avgSpline.index,corrector.avgSpline['xS'])
plt.plot(np.arange(100,50000), corrector.splines['xS'][0](np.arange(100,50000)) - x0)
plt.show()

In [ ]:
# BE SURE TO DROP NULLS FIRST
df3.dropna(inplace = True)
df3.to_csv('fullData.csv', index = False)

In [ ]:
df5 = df2[df2['frame'] > 4000]

In [ ]:
df5.to_csv('partialData.csv', index = False)

In [ ]:
df2.dropna(inplace = True)